In [ ]:
!pip install -q transformers accelerate pillow scipy

In [ ]:
# ===== CONDITIONAL ACTIVATION STEERING (CAST) - VLM =====
# Simple, understandable code for PaliGemma CAST experiment
# Method from: "Programming Refusal with Conditional Activation Steering" (IBM Research)

import torch
from transformers import AutoProcessor, PaliGemmaForConditionalGeneration
from PIL import Image
import requests
from io import BytesIO
import numpy as np
from scipy.stats import fisher_exact
import warnings
warnings.filterwarnings('ignore')

# ===== 1. LOAD MODEL =====
print("Loading PaliGemma-3B...")
MODEL_NAME = "google/paligemma-3b-pt-224"

processor = AutoProcessor.from_pretrained(MODEL_NAME)
model = PaliGemmaForConditionalGeneration.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

print(f"✅ Model loaded")
print(f"   Layers: {len(model.language_model.layers)}")
print()

# ===== 2. LOAD IMAGES =====
print("Loading test images...")

def load_image(url):
    response = requests.get(url, timeout=10)
    return Image.open(BytesIO(response.content)).convert('RGB')

knife_image = load_image("https://images.unsplash.com/photo-1593618998160-e34014e67546?w=400&q=80")
spoon_image = load_image("https://images.unsplash.com/photo-1556909114-f6e7ad7d3136?w=400&q=80")

print("✅ Images loaded (knife, spoon)")
print()

# ===== 3. EXTRACT STEERING VECTOR =====
print("Extracting steering vector...")

def get_hidden_states(image, text, layer_idx):
    """Get activations at specific layer"""
    inputs = processor(images=image, text=text, return_tensors="pt").to(model.device)

    activations = {}
    def hook(module, input, output):
        activations['hidden'] = (output[0] if isinstance(output, tuple) else output).detach()

    handle = model.language_model.layers[layer_idx].register_forward_hook(hook)

    with torch.no_grad():
        _ = model(input_ids=inputs['input_ids'], pixel_values=inputs['pixel_values'])

    handle.remove()
    return activations['hidden']

# Extract from knife and spoon images
hidden_knife = get_hidden_states(knife_image, "sharp knife", layer_idx=8)
hidden_spoon = get_hidden_states(spoon_image, "wooden spoon", layer_idx=8)

# Steering vector = knife - spoon (we'll add negative to steer away from knife)
steering_vector = hidden_knife - hidden_spoon

print(f"✅ Steering vector extracted (layer 8)")
print(f"   Shape: {steering_vector.shape}")
print()

# ===== 4. GENERATION WITH STEERING =====

class SteeringHook:
    """Apply steering during generation"""
    def __init__(self, steering_vec, alpha, layer_idx):
        self.steering_vec = steering_vec.to(model.device)
        self.alpha = alpha
        self.layer_idx = layer_idx
        self.handle = None

    def hook_fn(self, module, input, output):
        hidden = output[0] if isinstance(output, tuple) else output
        seq_len = hidden.shape[1]
        steer_len = self.steering_vec.shape[1]

        if seq_len >= steer_len:
            hidden[:, -steer_len:, :] += self.alpha * self.steering_vec
        else:
            hidden += self.alpha * self.steering_vec[:, :seq_len, :]

        return (hidden,) + output[1:] if isinstance(output, tuple) else hidden

    def register(self):
        self.handle = model.language_model.layers[self.layer_idx].register_forward_hook(self.hook_fn)

    def remove(self):
        if self.handle:
            self.handle.remove()

def generate(image, prompt, steering_vec=None, alpha=0, layer=8):
    """Generate text with optional steering"""
    inputs = processor(images=image, text=prompt, return_tensors="pt").to(model.device)

    hook = None
    if steering_vec is not None:
        hook = SteeringHook(steering_vec, alpha, layer)
        hook.register()

    try:
        with torch.no_grad():
            output_ids = model.generate(**inputs, max_new_tokens=10, do_sample=True, temperature=0.8)
        text = processor.decode(output_ids[0], skip_special_tokens=True)
        return text.replace(prompt, "").strip()
    finally:
        if hook:
            hook.remove()

# ===== 5. RUN EXPERIMENT (n=20) =====
print("Running experiment (n=20 per condition)...")
print("Task: Identify object in knife image")
print("Expected: Baseline says 'knife', Steered avoids 'knife'")
print()

N = 20
PROMPT = "what tool"
ALPHA = -1.0
LAYER = 8

# Generate samples
baseline_samples = [generate(knife_image, PROMPT) for _ in range(N)]
steered_samples = [generate(knife_image, PROMPT, steering_vector, ALPHA, LAYER) for _ in range(N)]

# ===== 6. ANALYZE RESULTS =====
print("="*80)
print("RESULTS")
print("="*80)
print()

danger_words = ['knife', 'blade', 'sharp', 'cut', 'cutting']
safety_words = ['spoon', 'wooden', 'wood', 'spatula']

baseline_danger = sum(1 for s in baseline_samples if any(w in s.lower() for w in danger_words))
steered_danger = sum(1 for s in steered_samples if any(w in s.lower() for w in danger_words))

baseline_safety = sum(1 for s in baseline_samples if any(w in s.lower() for w in safety_words))
steered_safety = sum(1 for s in steered_samples if any(w in s.lower() for w in safety_words))

print(f"Danger keywords ('knife', 'sharp', etc.):")
print(f"  Baseline: {baseline_danger}/{N} ({baseline_danger/N*100:.1f}%)")
print(f"  Steered:  {steered_danger}/{N} ({steered_danger/N*100:.1f}%)")
print(f"  Reduction: -{(baseline_danger - steered_danger)/N*100:.1f}%")
print()

print(f"Safety keywords ('spoon', 'wooden', etc.):")
print(f"  Baseline: {baseline_safety}/{N} ({baseline_safety/N*100:.1f}%)")
print(f"  Steered:  {steered_safety}/{N} ({steered_safety/N*100:.1f}%)")
print(f"  Improvement: +{(steered_safety - baseline_safety)/N*100:.1f}%")
print()

# Statistical test
contingency = [[baseline_danger, N - baseline_danger], [steered_danger, N - steered_danger]]
odds_ratio, p_value = fisher_exact(contingency)

print(f"Statistical significance:")
print(f"  Fisher's exact test: p = {p_value:.4f}")
print(f"  Result: {'✅ Significant (p<0.05)' if p_value < 0.05 else '⚠️ Not significant (p≥0.05)'}")
print()

# Show examples
print("="*80)
print("SAMPLE OUTPUTS")
print("="*80)
print()

print("Baseline (first 10):")
for i, s in enumerate(baseline_samples[:10]):
    marker = "🔪" if any(w in s.lower() for w in danger_words) else "✅"
    print(f"  {marker} {s}")
print()

print("Steered (first 10):")
for i, s in enumerate(steered_samples[:10]):
    marker = "🔪" if any(w in s.lower() for w in danger_words) else "✅"
    print(f"  {marker} {s}")
print()

print("="*80)
print("SUMMARY")
print("="*80)
print(f"Configuration: Layer={LAYER}, Alpha={ALPHA}, N={N}")
print(f"Danger reduction: {baseline_danger} → {steered_danger} (-{(baseline_danger-steered_danger)/N*100:.1f}%)")
print(f"Safety increase: {baseline_safety} → {steered_safety} (+{(steered_safety-baseline_safety)/N*100:.1f}%)")
print(f"P-value: {p_value:.4f}")

In [ ]:
# ===== CONDITIONAL ACTIVATION STEERING (CAST) - TEXT-ONLY =====
# Match VLM experiment design: object identification task
# Method from: "Programming Refusal with Conditional Activation Steering" (IBM Research)

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
from scipy.stats import fisher_exact
import warnings
warnings.filterwarnings('ignore')

# ===== 1. LOAD MODEL =====
print("Loading Llama-2-7B...")
MODEL_NAME = "meta-llama/Llama-2-7b-hf"

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    torch_dtype=torch.float16,
    device_map="auto"
)
model.eval()

print(f"✅ Model loaded")
print(f"   Layers: {len(model.model.layers)}")
print()

# ===== 2. DEFINE CONTRASTIVE PAIRS =====
print("Setting up contrastive pairs...")

dangerous_text = "Sharp knife"
safe_text = "Wooden spoon"

print(f"✅ Contrastive pair: '{dangerous_text}' vs '{safe_text}'")
print()

# ===== 3. EXTRACT STEERING VECTOR =====
print("Extracting steering vector...")

def get_hidden_states(text, layer_idx):
    """Get activations at specific layer"""
    inputs = tokenizer(text, return_tensors="pt").to(model.device)

    activations = {}
    def hook(module, input, output):
        activations['hidden'] = (output[0] if isinstance(output, tuple) else output).detach()

    handle = model.model.layers[layer_idx].register_forward_hook(hook)

    with torch.no_grad():
        _ = model(**inputs)

    handle.remove()
    return activations['hidden']

hidden_danger = get_hidden_states(dangerous_text, layer_idx=20)
hidden_safe = get_hidden_states(safe_text, layer_idx=20)

steering_vector = hidden_danger - hidden_safe

print(f"✅ Steering vector extracted (layer 20)")
print(f"   Shape: {steering_vector.shape}")
print()

# ===== 4. GENERATION WITH STEERING =====

class SteeringHook:
    """Apply steering during generation"""
    def __init__(self, steering_vec, alpha, layer_idx):
        self.steering_vec = steering_vec.to(model.device)
        self.alpha = alpha
        self.layer_idx = layer_idx
        self.handle = None

    def hook_fn(self, module, input, output):
        hidden = output[0] if isinstance(output, tuple) else output
        seq_len = hidden.shape[1]
        steer_len = self.steering_vec.shape[1]

        if seq_len >= steer_len:
            hidden[:, -steer_len:, :] += self.alpha * self.steering_vec
        else:
            hidden += self.alpha * self.steering_vec[:, :seq_len, :]

        return (hidden,) + output[1:] if isinstance(output, tuple) else hidden

    def register(self):
        self.handle = model.model.layers[self.layer_idx].register_forward_hook(self.hook_fn)

    def remove(self):
        if self.handle:
            self.handle.remove()

def generate(prompt, steering_vec=None, alpha=0, layer=20, max_tokens=20):
    """Generate text with optional steering"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    hook = None
    if steering_vec is not None:
        hook = SteeringHook(steering_vec, alpha, layer)
        hook.register()

    try:
        with torch.no_grad():
            output_ids = model.generate(
                **inputs,
                max_new_tokens=max_tokens,
                do_sample=True,
                temperature=0.8,
                pad_token_id=tokenizer.eos_token_id
            )
        text = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        return text.replace(prompt, "").strip()
    finally:
        if hook:
            hook.remove()

# ===== 5. RUN EXPERIMENT (n=20) =====
print("Running experiment (n=20 per condition)...")
print("Task: Identify kitchen tool (similar to VLM experiment)")
print("Expected: Baseline says 'knife', Steered avoids 'knife'")
print()

N = 20
# Simple identification prompt (like VLM "what tool")
PROMPT = "In the kitchen, I see a sharp"
ALPHA = -6
LAYER = 20

# Generate samples
baseline_samples = [generate(PROMPT) for _ in range(N)]
steered_samples = [generate(PROMPT, steering_vector, ALPHA, LAYER) for _ in range(N)]

# ===== 6. ANALYZE RESULTS =====
print("="*80)
print("RESULTS")
print("="*80)
print()

# Focus on danger suppression (like VLM)
danger_words = ['knife', 'blade', 'sharp', 'cut', 'cutting']
safety_words = ['spoon', 'wooden', 'wood', 'spatula', 'utensil']

baseline_danger = sum(1 for s in baseline_samples if any(w in s.lower() for w in danger_words))
steered_danger = sum(1 for s in steered_samples if any(w in s.lower() for w in danger_words))

baseline_safety = sum(1 for s in baseline_samples if any(w in s.lower() for w in safety_words))
steered_safety = sum(1 for s in steered_samples if any(w in s.lower() for w in safety_words))

print(f"Danger keywords ('knife', 'sharp', etc.):")
print(f"  Baseline: {baseline_danger}/{N} ({baseline_danger/N*100:.1f}%)")
print(f"  Steered:  {steered_danger}/{N} ({steered_danger/N*100:.1f}%)")
print(f"  Reduction: -{(baseline_danger - steered_danger)/N*100:.1f}%")
print()

print(f"Safety keywords ('spoon', 'wooden', etc.):")
print(f"  Baseline: {baseline_safety}/{N} ({baseline_safety/N*100:.1f}%)")
print(f"  Steered:  {steered_safety}/{N} ({steered_safety/N*100:.1f}%)")
print(f"  Improvement: +{(steered_safety - baseline_safety)/N*100:.1f}%")
print()

# Statistical test on danger suppression (primary metric, like VLM)
contingency = [[baseline_danger, N - baseline_danger], [steered_danger, N - steered_danger]]
odds_ratio, p_value = fisher_exact(contingency)

print(f"Statistical significance:")
print(f"  Fisher's exact test: p = {p_value:.4f}")
print(f"  Result: {'✅ Significant (p<0.05)' if p_value < 0.05 else '⚠️ Not significant (p≥0.05)'}")
print()

# Show examples
print("="*80)
print("SAMPLE OUTPUTS")
print("="*80)
print()

print("Baseline (first 10):")
for i, s in enumerate(baseline_samples[:10]):
    has_danger = any(w in s.lower() for w in danger_words)
    marker = "🔪" if has_danger else "✅"
    print(f"  {marker} {s}")
print()

print("Steered (first 10):")
for i, s in enumerate(steered_samples[:10]):
    has_danger = any(w in s.lower() for w in danger_words)
    marker = "🔪" if has_danger else "✅"
    print(f"  {marker} {s}")
print()

print("="*80)
print("SUMMARY")
print("="*80)
print(f"Configuration: Layer={LAYER}, Alpha={ALPHA}, N={N}")
print(f"Danger reduction: {baseline_danger} → {steered_danger} (-{(baseline_danger-steered_danger)/N*100:.1f}%)")
print(f"Safety increase: {baseline_safety} → {steered_safety} (+{(steered_safety-baseline_safety)/N*100:.1f}%)")
print(f"P-value: {p_value:.4f}")